In [ ]:
import sys
sys.path.append('../')
from functions.functions import count_SIR, STFT1024, calculate_SIR_for_frames, get_SIR_of_iFICA_result, create_ideal_pilot_from_SIR

import numpy as np
import soundfile as sf
import pandas as pd
import plotly.graph_objects as go

In [2]:
number_of_samples = 2000

SIR_original = np.zeros((number_of_samples, 1))
SIR_SOI_pilot = np.zeros((number_of_samples, 1))
SIR_impovement_SOI_pilot = np.zeros((number_of_samples, 1))
SIR_label_pilot = np.zeros((number_of_samples, 1))
SIR_improvement_label_pilot = np.zeros((number_of_samples, 1))
SIR_ones_pilot = np.zeros((number_of_samples, 1))
SIR_improvement_no_pilot = np.zeros((number_of_samples, 1))
SIR_X_pilot = np.zeros((number_of_samples, 1))
SIR_improvement_X_pilot = np.zeros((number_of_samples, 1))

In [ ]:
for i in range(0, number_of_samples):
    s, fs_s = sf.read(f'../data/train/s{i}.wav')
    y, fs_y = sf.read(f'../data/train/y{i}.wav')

    nr_of_mics = 9
    
    mix_signal = s + y
    
    SIR_original[i], _ = count_SIR(s, y)
    
    X = STFT1024(mix_signal.T).transpose(0,2,1)
    SOI = STFT1024(s.T).transpose(0,2,1)
    INT = STFT1024(y.T).transpose(0,2,1)
    
    d, N, K = X.shape

    # PILOT IDEALNI ZE SOI
    pilot = np.sum(np.abs(SOI[0,:,:])**2, axis=1)
    SIR_SOI_pilot[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot)
    SIR_impovement_SOI_pilot[i] = SIR_SOI_pilot[i] - SIR_original[i]

    # PILOT IDEALNI ZE SIR
    sir_values_per_frame = calculate_SIR_for_frames(SOI.transpose(0,2,1), INT.transpose(0,2,1))
    pilot_SIR = create_ideal_pilot_from_SIR(sir_values_per_frame)
    SIR_label_pilot[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot_SIR)
    SIR_improvement_label_pilot[i] = SIR_label_pilot[i] - SIR_original[i]

    # PILOT Z JEDNICEK
    pilot_ones = np.ones_like(pilot)
    SIR_ones_pilot[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot_ones)
    SIR_improvement_no_pilot[i] = SIR_ones_pilot[i] - SIR_original[i]

    # PILOT Z MIXOVANEHO SIGNALU
    pilot_X = np.sum(np.abs(X[0,:,:])**2, axis=1)
    SIR_X_pilot[i] = get_SIR_of_iFICA_result(nr_of_mics, K, X, SOI, INT, pilot_X)
    SIR_improvement_X_pilot[i] = SIR_X_pilot[i] - SIR_original[i]


    print(i)

In [ ]:
SIR_df = pd.DataFrame({
    'SIR_original': SIR_original.flatten(),
    'SIR_SOI_pilot': SIR_SOI_pilot.flatten(),
    'imp_SIR_SOI_pilot': SIR_impovement_SOI_pilot.flatten(),
    'SIR_label_pilot': SIR_label_pilot.flatten(),
    'imp_SIR_label_pilot': SIR_improvement_label_pilot.flatten(),
    'SIR_ones_pilot': SIR_ones_pilot.flatten(),
    'imp_SIR_ones_pilot': SIR_improvement_no_pilot.flatten(),
    'SIR_X_pilot': SIR_X_pilot.flatten(),
    'imp_SIR_X_pilot': SIR_improvement_X_pilot.flatten()
})

SIR_df

In [12]:
print(f"Průměrné zlepšení {np.mean(SIR_df["imp_SIR_SOI_pilot"])}")
print(f"Průměrné zlepšení s pilotem z label {np.mean(SIR_df["imp_SIR_label_pilot"])}")
print(f"Průměrné zlepšení bez pilota {np.mean(SIR_df["imp_SIR_ones_pilot"])}")
print(f"Průměrné zlepšení s pilotem z X {np.mean(SIR_df["imp_SIR_X_pilot"])}")

Průměrné zlepšení 6.051024399974921
Průměrné zlepšení s pilotem z label 6.41137451337926
Průměrné zlepšení bez pilota 3.5100064795728922
Průměrné zlepšení s pilotem z X 3.922005875840928


In [11]:
SIR_df.to_csv('../results/SIR_train.csv', index=True)

In [13]:
SIR_df

,SIR_original,SIR_SOI_pilot,imp_SIR_SOI_pilot,SIR_ones_pilot,imp_SIR_ones_pilot,SIR_X_pilot,imp_SIR_X_pilot,SIR_label_pilot,imp_SIR_label_pilot
0,0.043753,5.035898,4.992145,3.730764,3.687011,3.745241,3.701488,4.897083,4.853329
1,2.365760,7.792521,5.426761,7.300645,4.934886,7.245115,4.879355,8.086943,5.721183
2,2.125906,10.518184,8.392278,9.429163,7.303257,9.391805,7.265899,10.300209,8.174303
3,1.288873,8.405974,7.117100,8.425883,7.137009,5.383518,4.094644,9.113372,7.824499
4,2.263182,7.438798,5.175616,2.749002,0.485820,2.494578,0.231396,7.745416,5.482234
...,...,...,...,...,...,...,...,...,...
1995,-0.035784,4.417775,4.453559,0.647126,0.682910,-2.633125,-2.597341,6.613839,6.649623
1996,0.526029,11.279239,10.753210,5.497952,4.971923,3.075962,2.549933,10.967152,10.441123
1997,4.830665,17.038235,12.207570,16.475313,11.644648,15.425203,10.594537,16.205641,11.374976
1998,4.490863,8.804493,4.313630,7.685954,3.195091,7.912196,3.421333,9.361797,4.870935


In [ ]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_SOI_pilot"],
    name="Zlepšení SIR pro ideální pilot"
))


fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_label_pilot"],
    name="Zlepšení SIR pro klasifikovaný pilot"
))

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_ones_pilot"],
    name="Zlepšení SIR pro jednotkový pilot"
))

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_X_pilot"],
    name="Zlepšení SIR pro pilot z původního signálu"
))

fig_box.update_layout(
    title="Boxplot zlepšení SIR",
    yaxis_title="Hodnoty zlepšení SIR",
    legend=dict(title="Legenda")
)

fig_box.show()

In [20]:
fig_box = go.Figure()

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_SOI_pilot"],
    name="Ideální pilot (energie SOI)"
))

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_label_pilot"],
    name="Ideální pilot (intenzity, labely)"
))

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_X_pilot"],
    name="Pilot z energie původního signálu"
))

fig_box.add_trace(go.Box(
    y=SIR_df["imp_SIR_ones_pilot"],
    name="Pilot z 1 (slepá extrakce)"
))

fig_box.update_layout(
        title=dict(
        text="Zlepšení SIR extrahovaného signálu s různými piloty (trénovací data)",
        font=dict(size=20)
    ),
    xaxis=dict(
        title=dict(text="Typ pilotu", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    yaxis=dict(
        title=dict(text="Zlepšení SIR [dB]", font=dict(size=18)),
        tickfont=dict(size=16)
    ),
    showlegend=False,
    height=800,
    width=800
)

fig_box.show()
